In [71]:
# !pip install google-auth
# !pip install requests

In [72]:
# %pip install openai
# %pip install git+https://github.com/openai/swarm.git


In [73]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = 'my-project-0004-346516'

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [74]:
import vertexai
import openai

from google.auth import default, transport

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Programmatically get an access token
credentials, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

# # OpenAI Client
client = openai.OpenAI(
    base_url=f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
    api_key=credentials.token,
)

response = client.chat.completions.create(
    model="google/gemini-1.5-flash-002",
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
)

print(response.choices[0].message.content)
# Example response:
# The sky is blue due to a phenomenon called **Rayleigh scattering**.
# Sunlight is made up of all the colors of the rainbow.
# As sunlight enters the Earth's atmosphere ...

The sky is blue due to a phenomenon called **Rayleigh scattering**.  Sunlight is made up of all the colors of the rainbow.  When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).  These molecules are much smaller than the wavelengths of visible light.

Rayleigh scattering affects shorter wavelengths of light more strongly than longer wavelengths.  Blue and violet light have the shortest wavelengths, so they are scattered much more than red and orange light.  This scattered blue light is what we see when we look at the sky.

While violet light is actually scattered *more* than blue light, our eyes are more sensitive to blue, and the sun emits slightly less violet light, resulting in a blue sky rather than a violet one.



In [80]:
print(credentials.token)

ya29.a0AeDClZC5nRCoiNvSPvfZgCzFNiCjdJ3fuiToXiq-f1GCN14Sgf7D9d4a2_x1K4wK-UQ2srSHiAji5XxBp0DfobiPul-0rLoLxVRth-BwOhwlXbvy7qOkvZ4G76A_Ab7fBvlbH16aArCrl4iR40RDMyUeHp7X-f6gZa87mfNbGgaCgYKAScSARESFQHGX2Mi668a9__bOsZVNd7Bw81N-A0177


### Swarm config

In [75]:

from openai import OpenAI
from swarm import Swarm, Agent
import gradio as gr

# Initialize Swarm client
ollama_client = OpenAI(
    base_url=f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
    api_key=credentials.token, 
)


In [76]:
from swarm import Swarm, Agent
import os

In [77]:

# Define agent functions
def get_weather(location, time="now"):
    # Function to get weather information
    return f"The weather in {location} at {time} is sunny but it can change quickly."
def send_email(recipient, subject, body):
    # Function to send an email
    return "Email sent successfully."
# Define agents
weather_agent = Agent(
    name="Weather Agent",
    instructions="Provide weather updates.",
    functions=[get_weather, send_email],
    model="google/gemini-1.5-flash-002",
)
# Initialise Swarm client and run conversation
# client = Swarm()
client = Swarm(client=ollama_client)

response = client.run(
    agent=weather_agent,
    messages=[{"role": "user", "content": "What's the weather in Singapore ?"}],
)

print(response.messages[-1]["content"])

The weather in Singapore right now is sunny, but it can change quickly.



In [78]:

english_agent = Agent(
    name="English Agent",
    instructions="You only speak English. transfer to other agent if needed",
    model="google/gemini-1.5-flash-002"    
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
    model="google/gemini-1.5-flash-002",
)


def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent


english_agent.functions.append(transfer_to_spanish_agent)

messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

I'm sorry, I only speak English.  Could you please rephrase your question in English?



In [79]:
messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=spanish_agent, messages=messages)

print(response.messages[-1]["content"])

¡Hola! Bien, gracias, ¿y tú?

